<a href="https://colab.research.google.com/github/raja-jamwal/-medicine-notes-segmenter/blob/main/Part_1_The_Foundation_Prompt_Chaining%2C_Routing%2C_and_Parallelization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Agents in Python and n8n in 2026
Companion to https://rajajamwal.substack.com/p/building-agents-in-python-and-n8n

Subscribe to my blog, https://rajajamwal.substack.com

Before we get to complex autonomous agents, we must master the flow of data.

We will cover:
1.  **Prompt Chaining:** Breaking complex tasks into sequential steps.
2.  **Routing:** Adding conditional logic to direct requests to the right expert.
3.  **Parallelization:** Running independent tasks simultaneously for speed.

### The Stack
*   **Python**
*   **LangChain (LCEL):** For orchestration.
*   **OpenAI:** As our LLM provider.

### The n8n Connection
If you are following along with **n8n**:
*   **Chaining** = Connecting nodes linearly.
*   **Routing** = The `Switch` or `If` Node.
*   **Parallelization** = Connecting one node to multiple outputs + `Merge` Node.

In [2]:
# @title 1. Install Dependencies
# We need langchain and the openai integration
!pip install -qU langchain langchain-openai langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 13.4 MB/s eta 0:00:00


In [3]:
import os
from getpass import getpass

# @title 2. Setup API Key
# Enter your OpenAI API Key when prompted
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

# Initialize the Model
from langchain_openai import ChatOpenAI

# We use a temperature of 0 for consistent, deterministic results
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("✅ Environment Setup Complete.")

Enter your OpenAI API Key: ··········
✅ Environment Setup Complete.


## Pattern 1: Prompt Chaining (The Pipeline)

**The Problem:** Asking an LLM to do too much in one prompt (e.g., "Research this company, write a summary, and translate it to Spanish") often leads to hallucinations or missed instructions.

**The Solution:** Break the task into a sequence. The output of Step 1 becomes the input of Step 2.

**The Scenario:** We want to generate a **Company Name** based on a product description, and then write a **Slogan** for that specific name.

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- Step 1: Generate Company Name ---
name_prompt = ChatPromptTemplate.from_template(
    "What is a catchy, futuristic company name for a startup that makes: {product}? Return ONLY the name."
)
# Chain 1: Input -> Prompt -> LLM -> String Output
name_chain = name_prompt | llm | StrOutputParser()

# --- Step 2: Generate Slogan ---
slogan_prompt = ChatPromptTemplate.from_template(
    "Write a 3-word slogan for a company named '{company_name}'."
)
slogan_chain = slogan_prompt | llm | StrOutputParser()

# --- The Full Chain (LCEL) ---
# We pipe the output of name_chain into the 'company_name' variable for the slogan_chain
overall_chain = (
    {"company_name": name_chain}
    | slogan_chain
)

# --- Execution ---
product_description = "A coffee machine that uses AI to predict exactly what you want to drink"
result = overall_chain.invoke({"product": product_description})

print(f"Input Product: {product_description}")
print(f"Final Result (Slogan): {result}")

Input Product: A coffee machine that uses AI to predict exactly what you want to drink
Final Result (Slogan): "Crafting Coffee Innovation."


## Pattern 2: Routing (The Decision Maker)

**The Problem:** A single prompt cannot handle every type of user request efficiently. You don't want a "Creative Writer" model handling "Math" questions.

**The Solution:** Use a **Router** step to classify the intent, then direct the flow to a specialized chain.

**The Scenario:** We are building a support bot.
*   If the user asks about **Math/Logic**, route to a "Logic Chain".
*   If the user asks for **Creative Writing**, route to a "Creative Chain".

In [5]:
from langchain_core.runnables import RunnableBranch, RunnablePassthrough

# --- Define the Specialized Chains ---

# 1. Math Chain (Strict, concise)
math_chain = (
    ChatPromptTemplate.from_template("You are a mathematician. Solve this: {input}")
    | llm
    | StrOutputParser()
)

# 2. Creative Chain (Flowery, descriptive)
creative_chain = (
    ChatPromptTemplate.from_template("You are a poet. Write about: {input}")
    | llm
    | StrOutputParser()
)

# --- The Router ---
# First, we ask the LLM to classify the input
classification_chain = (
    ChatPromptTemplate.from_template(
        "Classify the following input as either 'MATH' or 'CREATIVE'. Return ONLY the word.\nInput: {input}"
    )
    | llm
    | StrOutputParser()
)

# Define the Branching Logic
branch = RunnableBranch(
    (lambda x: "MATH" in x["topic"], math_chain),      # If topic is MATH, go to math_chain
    (lambda x: "CREATIVE" in x["topic"], creative_chain), # If topic is CREATIVE, go to creative_chain
    creative_chain # Default fallback
)

# --- The Full Routing Chain ---
full_chain = (
    {"topic": classification_chain, "input": RunnablePassthrough()}
    | branch
)

# --- Execution ---
print("--- Test 1: Math ---")
print(full_chain.invoke({"input": "What is the square root of 144?"}))

print("\n--- Test 2: Creative ---")
print(full_chain.invoke({"input": "The sunset over the ocean"}))

--- Test 1: Math ---
The square root of 144 is 12.

--- Test 2: Creative ---
In the hush of evening's gentle sigh,  
The sun descends, a fiery eye,  
Casting gold upon the waves,  
Where whispers of the ocean braves.  

A canvas stretched, the sky ablaze,  
With hues of crimson, orange, and mauve,  
Each brushstroke dances, a fleeting phase,  
As day concedes to night’s soft grove.  

The horizon swallows the molten sphere,  
A liquid ember, drawing near,  
While waves, like secrets, lap the shore,  
In rhythmic tales of ancient lore.  

The salty breeze, a lover's breath,  
Caresses skin, a sweet caress,  
As shadows stretch and daylight wanes,  
The world transforms, yet still remains.  

In this moment, time stands still,  
The heart is full, the spirit thrills,  
For in the sunset’s warm embrace,  
We find our peace, our sacred space.  

So let us linger, hand in hand,  
As twilight weaves its magic strand,  
For in the sunset over the sea,  
We glimpse eternity, wild and free.  


## Pattern 3: Parallelization (The Efficiency Booster)

**The Problem:** Sometimes a task requires multiple independent perspectives (e.g., "Pros" and "Cons"). Running them one after another (sequentially) is slow.

**The Solution:** Run them at the same time (in parallel) and merge the results at the end.

**The Scenario:** We are analyzing a product. We want to generate a list of **Pros** and a list of **Cons** simultaneously, then combine them into a final review.

In [6]:
from langchain_core.runnables import RunnableParallel

# --- Define Independent Tasks ---

# Task 1: Identify Pros
pros_chain = (
    ChatPromptTemplate.from_template("List 3 distinct PROS of: {product}")
    | llm
    | StrOutputParser()
)

# Task 2: Identify Cons
cons_chain = (
    ChatPromptTemplate.from_template("List 3 distinct CONS of: {product}")
    | llm
    | StrOutputParser()
)

# --- The Parallel Map ---
# RunnableParallel runs these keys at the same time
map_chain = RunnableParallel(
    pros=pros_chain,
    cons=cons_chain
)

# --- The Final Synthesis ---
# Combines the parallel outputs into one final summary
synthesis_chain = (
    ChatPromptTemplate.from_template(
        "Combine these into a balanced review:\n\nPROS:\n{pros}\n\nCONS:\n{cons}"
    )
    | llm
    | StrOutputParser()
)

# --- Full Chain ---
parallel_workflow = map_chain | synthesis_chain

# --- Execution ---
topic = "Remote Work"
result = parallel_workflow.invoke({"product": topic})

print(f"Topic: {topic}\n")
print(result)

Topic: Remote Work

**Balanced Review of Remote Work**

Remote work has become increasingly popular, offering a mix of advantages and challenges that can significantly impact both employees and employers. 

**Pros:**

One of the most notable benefits of remote work is the **flexibility and work-life balance** it provides. Employees can often set their own schedules, allowing them to manage personal responsibilities—such as childcare or errands—while still meeting professional obligations. This flexibility can lead to increased job satisfaction and overall well-being.

Another significant advantage is the **reduction in commuting time and costs**. By eliminating daily travel, employees save both time and money, which can alleviate stress and provide more opportunities to focus on work or personal activities. This aspect of remote work can enhance productivity and contribute positively to an employee's quality of life.

For employers, remote work expands the **access to a broader talent 

## Summary

Congratulations! You have just implemented the three fundamental patterns of Agentic AI:

1.  **Chaining:** `A -> B` (Sequential)
2.  **Routing:** `If X -> A, Else -> B` (Conditional)
3.  **Parallelization:** `A + B -> C` (Simultaneous)

In **Part 2**, we will make our agents smarter by introducing **Reflection** (Self-Correction) and **Tool Use** (Connecting to the outside world)